In [3]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

import sys
sys.path.append("../")
import os

import re
import datetime
import pandas as pd
import pandas_market_calendars as mcal
from polygon import RESTClient
#from tqdm.notebook import tqdm
import time
import pytz
eastern = 'US/Eastern'

In [23]:
key = "qRFktbNL6A_WzaFhqR80IllApsmjPY6a"

def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def get_split_data_for_stock(stock):
    with RESTClient(key) as client:
        try:
            respex = client.reference_stock_splits(str(stock).upper())
            result = respex.__dict__['results']
            return result
        except:
            print("Error getting split data for "+ str(stock))
            return []

def get_dividend_data_for_stock(stock):
    with RESTClient(key) as client:
        try:
            respex = client.reference_stock_dividends(str(stock).upper())
            result = respex.__dict__['results']
            return result
        except:
            print("Error getting dividend data for "+ str(stock))
            return []
        
def extract_qualified_symbols(df, plvl, advlvl):
    qualified = df[(df.p_lvl == plvl) & (df.adv_lvl == advlvl)]
    return qualified

In [24]:
def get_folder_list():    
    path = '.'
    directory_contents = os.listdir(path)

    r = re.compile(".*pv_adv_[0-9]_[0-9]")
    folder_list = list(filter(r.match, directory_contents)) # Read Note
    return folder_list

def get_all_files_in_folder(folder):
    return os.listdir(folder)

def extract_date_from_filename(filename):
    split_a = filename.split('trades-')
    split_b = split_a[1].split('.')
    return split_b[0]

def update_all_files():
    a = get_folder_list()
    count = 0
    
    all_file_dfs = []
    
    for folder in tqdm(a, desc="folders"):
        all_files = get_all_files_in_folder(folder)
        count = count + 1
        
        for file in tqdm(all_files, desc="Folder Files"):
            file_date = extract_date_from_filename(file)
            file_df = pd.read_csv(folder+'/'+str(file))
            file_df['trade_date'] = file_date
            file_df['group'] = count
            file_df['id'] = file_df[['trade_date', 'symbol']].sum(axis=1).map(hash)            
            file_df['split_ex_date'] = "None"
            file_df['div_ex_date'] = "None"
            file_df['split_ratio'] = 1
            file_df['dividend_ratio'] = 0
            file_df.set_index("id", inplace=True)
            
            all_file_dfs.append(file_df)
            
    result_1 = pd.DataFrame().append(all_file_dfs)
    return result_1

def update_all_etf_files():

    all_file_dfs = []    
    for file in tqdm(all_etf_files, desc="etf Files"):
        file_date = extract_date_from_filename(file)
        file_df = pd.read_csv("pv_adv_etf_data"+'/'+str(file))
        file_df['trade_date'] = file_date
        file_df['id'] = file_df[['trade_date', 'symbol']].sum(axis=1).map(hash)            
        file_df.set_index("id", inplace=True)

        all_file_dfs.append(file_df)
            
    result_1 = pd.DataFrame().append(all_file_dfs)
    return result_1

In [25]:
main_df = pd.read_csv('stocks-profile.csv')
q_sym_list = []
for i in range(5):
    q_sym = extract_qualified_symbols(main_df, i, 2)
    q_sym_list = q_sym_list + list(q_sym['symbol'])

In [26]:
splits_df = []
dividend_df = []

In [27]:
for i in tqdm(range(len(q_sym_list))):
    
    ticker = q_sym_list[i]
    
    sp = get_split_data_for_stock(ticker)
    div = get_dividend_data_for_stock(ticker)
    a = pd.DataFrame(sp)
    b = pd.DataFrame(div)
   
    splits_df.append(a)
    
    dividend_df.append(b)
    
    time.sleep(0.1)

In [28]:
dividend_df = pd.concat(dividend_df)
splits_df = pd.concat(splits_df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [33]:
splits_df.head()

,declaredDate,exDate,forfactor,paymentDate,ratio,ticker,tofactor
0,2016-11-09,2016-11-09,1.0,2016-11-09,3.00,GNUS,3.0
0,NaN,2000-04-11,NaN,2000-04-11,0.25,NOK,NaN
1,NaN,1999-04-12,NaN,1999-04-12,0.50,NOK,NaN
2,NaN,1998-04-16,NaN,1998-04-16,0.50,NOK,NaN
0,2020-08-10,2020-08-10,1.0,2020-08-10,25.00,TOPS,25.0


In [37]:
spl_col = list(splits_df.columns)
divd_cols = list(dividend_df.columns)

for i,s in enumerate(spl_col):
    spl_col[i] = "split_"+s
    
for i,s in enumerate(divd_cols):
    divd_cols[i] = "dividend_"+s

In [38]:
spl_df_copy = splits_df.copy()
divd_df_copy = dividend_df.copy()

In [39]:
spl_df_copy.columns = spl_col
divd_df_copy.columns = divd_cols

In [40]:
spl_div = pd.concat([spl_df_copy, divd_df_copy])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [42]:
spl_div.to_csv('split_and_divided_top_5.csv')

In [ ]:
ouut = update_all_files()

In [ ]:
all_etf_files_df = update_all_etf_files()

In [ ]:
all_etf_files = os.listdir('pv_adv_etf_data')

In [ ]:
for index, row in tqdm(df.iterrows(), desc='Symbols', total=len(df)):
    
    for item in list(row['splits']):
        ex_date = item['exDate']

        payment_date = item['paymentDate']
        ratio = item['ratio']
        symbol = item['ticker']

        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'split_ex_date'] = ex_date
        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'split_ratio'] = ratio

In [ ]:
for index, row in tqdm(symbols_with_dividends.iterrows(), desc='Symbols', total=len(symbols_with_dividends)):
    
    for item in list(row['dividends']):
        ex_date = item['exDate']

        payment_date = item['paymentDate']
        ratio = item['amount']
        symbol = item['ticker']

        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'div_ex_date'] = ex_date
        new_file_df.loc[(new_file_df['symbol'] == symbol) & (new_file_df['trade_date'] == ex_date), 'dividend_ratio'] = ratio